In [9]:
# backend/embeddings.py
from sentence_transformers import SentenceTransformer
import numpy as np

class EmbeddingManager:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)
        print(f"✅ Loaded embedding model: {model_name}")

    def generate_embeddings(self, texts):
        """Generate embeddings for a list of text chunks"""
        return np.array(self.model.encode(texts, show_progress_bar=True))


In [10]:
# backend/store.py
import os
import chromadb
from chromadb.config import Settings

class VectorStore:
    def __init__(self, path="../data/vector_store"):
        self.client = chromadb.PersistentClient(path=path, settings=Settings(anonymized_telemetry=False))
        self.collection = self.client.get_or_create_collection(name="ece_concepts")
        print(f"✅ Vector store initialized at {path}")

    def add_documents(self, docs, embeddings):
        """Store docs with embeddings"""
        ids = [str(i) for i in range(len(docs))]
        self.collection.add(documents=docs, embeddings=embeddings.tolist(), ids=ids)
        print(f"🧠 Stored {len(docs)} chunks in vector DB.")

    def retrieve_similar_docs(self, query_embedding, top_k=3):
        """Retrieve top-k relevant chunks"""
        results = self.collection.query(query_embeddings=[query_embedding.tolist()], n_results=top_k)
        docs = results["documents"][0]
        return docs


In [12]:
# backend/rag_app.py
import os
from openai import OpenAI
from embeddings import EmbeddingManager
from store import VectorStore

class RAGApp:
    def __init__(self):
        self.embedder = EmbeddingManager()
        self.vectorstore = VectorStore()
        # Load your key securely (from env variable)
        self.client = OpenAI(api_key=os.getenv("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"))

    def add_notes(self, text):
        chunks = [text[i:i+1000] for i in range(0, len(text), 800)]
        embeddings = self.embedder.generate_embeddings(chunks)
        self.vectorstore.add_documents(chunks, embeddings)
        return len(chunks)

    def ask(self, query):
        q_embed = self.embedder.generate_embeddings([query])[0]
        docs = self.vectorstore.retrieve_similar_docs(q_embed, top_k=3)
        context = "\n\n".join(docs)
        prompt = f"Use the following context to answer:\n{context}\n\nQuestion: {query}\nAnswer:"

        # Call OpenAI-compatible API
        response = self.client.chat.completions.create(
            model="gpt-oss-20b",  # your model name
            messages=[
                {"role": "system", "content": "You are an AI tutor specialized in ECE concepts."},
                {"role": "user", "content": prompt},
            ],
        )
        return response.choices[0].message.content


ModuleNotFoundError: No module named 'openai'